In [1]:
import pandas as pd
import numpy as np
import random
df=pd.read_csv('tiempos_por_tecla.csv')
df = df.drop(df[df.modo != 1].index)
df
prueba = ['usuario','practica']

In [2]:
df.head(10)
# sacar promedio de mantener la letra pulsada

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo
0,1,1,1,69,43,0,0,0,1
1,2,1,1,83,237,157,69,0,1
2,3,1,1,84,458,330,83,157,1
3,4,1,1,65,571,426,84,330,1
4,5,1,1,32,650,572,65,426,1
5,6,1,1,69,778,714,32,572,1
6,7,1,1,83,970,858,69,714,1
7,8,1,1,32,1066,986,83,858,1
8,9,1,1,85,1226,1146,32,986,1
9,10,1,1,78,1418,1338,85,1146,1


In [3]:
df['tiempo'] = df.down - df.anterior_down
df['anterior_up'] = df.up.shift(periods = 1)
df['anterior_up'] = np.where(df['down'] == 0, 0, df['anterior_up'])
df['solapa'] = np.where((df.anterior_up - df.down) > 0, 1, 0)
df['tecla'] = np.where(df.tecla == 32, 95, df.tecla)
df['anterior'] = np.where(df.anterior == 32, 95, df.anterior)
df['tecla'] = df['tecla'].apply(chr)
df['anterior'] = df['anterior'].apply(chr)
df['par'] = df.anterior + df.tecla
pt = df.groupby(by=['usuario','practica','par'])['tiempo'].median()
pt = pt.reset_index()
pt = pt.rename(columns={'tiempo':'mean'})
pa = df.groupby(by=['usuario'])['solapa'].sum()
pa = pa.reset_index()
pl = df.groupby(by=['usuario','practica','par'])['tiempo'].median()
pl = pl.reset_index()
pstd = df.groupby(by=['usuario','practica','par'])['tiempo'].std()
pstd = pl.reset_index()
pstd = pstd.rename(columns={'tiempo':'std'})
pstd

,index,usuario,practica,par,std
0,0,1,1, E,0.0
1,1,1,1,",_",895.5
2,2,1,1,.0,224.0
3,3,1,1,.1,320.0
4,4,1,1,.5,279.5
...,...,...,...,...,...
72081,72081,93,30,_Q,104.0
72082,72082,93,30,_S,135.0
72083,72083,93,30,_T,128.0
72084,72084,93,30,_U,156.0


In [4]:
pt

,usuario,practica,par,mean
0,1,1, E,0.0
1,1,1,",_",895.5
2,1,1,.0,224.0
3,1,1,.1,320.0
4,1,1,.5,279.5
...,...,...,...,...
72081,93,30,_Q,104.0
72082,93,30,_S,135.0
72083,93,30,_T,128.0
72084,93,30,_U,156.0


In [5]:
pl = pl.pivot(index=prueba, columns='par', values='tiempo')
pl = pl.reset_index()
pstd = pstd.pivot(index=prueba, columns='par', values='std')
pstd = pstd.reset_index()
pstd

par,usuario,practica, E,",_",.0,.1,.5,.E,04,15,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,0.0,895.5,224.0,320.0,279.5,1185.0,129.0,128.0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,2,0.0,200.0,353.0,255.0,264.5,1056.0,174.0,146.0,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
2,1,3,0.0,207.5,241.0,256.0,313.0,974.0,127.0,129.0,...,154.0,161.0,446.0,238.0,161.0,224.0,192.0,256.0,192.0,145.0
3,1,4,0.0,175.5,224.0,256.0,271.5,1281.0,112.0,143.0,...,328.0,176.0,150.0,320.0,129.0,192.0,159.0,288.0,175.5,184.0
4,1,5,0.0,176.0,609.0,223.0,464.5,1104.0,176.0,191.0,...,177.0,159.0,176.0,239.0,143.0,176.0,194.0,240.0,216.5,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,26,0.0,128.0,177.0,233.0,188.5,1088.0,223.0,71.0,...,200.0,152.0,176.0,144.0,90.0,154.0,192.0,112.0,123.5,131.5
538,93,27,0.0,172.5,224.0,222.0,184.0,664.0,200.0,121.0,...,160.0,200.0,128.0,104.0,120.0,152.0,775.0,88.0,163.5,211.5
539,93,28,0.0,100.5,176.0,191.0,220.0,696.0,216.0,105.0,...,412.0,192.0,127.0,89.0,80.0,129.0,143.0,80.0,132.5,159.5
540,93,29,0.0,116.5,175.0,200.0,200.0,872.0,226.0,96.0,...,136.0,152.0,168.0,176.0,80.0,144.0,408.0,96.0,140.5,144.0


In [7]:
pp = df.merge(pl, on=prueba, how='outer')
pp = pp.merge(pstd, on=prueba, how='outer',suffixes=('', 'm'))
#pp['lec'] = np.where((pp['mean'] + pp['std']) < pp['tiempo'], 1, 0)
pp

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,...,_Im,_Jm,_Lm,_Om,_Pm,_Qm,_Sm,_Tm,_Um,_Vm
0,1,1,1,E,43,0, ,0,1,0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,2,1,1,S,237,157,E,0,1,157,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
2,3,1,1,T,458,330,S,157,1,173,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
3,4,1,1,A,571,426,T,330,1,96,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
4,5,1,1,_,650,572,A,426,1,146,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136037,228727,64,39,A,52693,52574,U,52468,1,106,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5
136038,228728,64,39,R,52772,52670,A,52574,1,96,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5
136039,228729,64,39,I,52860,52756,R,52670,1,86,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5
136040,228730,64,39,O,52925,52828,I,52756,1,72,...,211.5,193.0,112.0,184.0,103.0,119.0,98.0,184.0,167.0,135.5


In [12]:
lis = df.par.unique() 
outliders
for k in lis:
    col = str(k) + 'out'
    pmean = str(k)
    pstd = str(k) + 'm'
    pp[col] = np.where((pp[pmean] + pp[pstd]) < pp['tiempo'], 1, 0)
    pp['out'] = 
pp

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,...,ALout,LIout,IDout,DAout,L_out,USout,SUout,UAout,IOout,O.out
0,1,1,1,E,43,0, ,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,S,237,157,E,0,1,157,...,0,0,0,0,0,0,0,0,1,0
2,3,1,1,T,458,330,S,157,1,173,...,0,0,0,0,0,0,1,0,1,0
3,4,1,1,A,571,426,T,330,1,96,...,0,0,0,0,0,0,0,0,1,0
4,5,1,1,_,650,572,A,426,1,146,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136037,228727,64,39,A,52693,52574,U,52468,1,106,...,0,0,0,0,0,0,0,0,0,0
136038,228728,64,39,R,52772,52670,A,52574,1,96,...,0,0,0,0,0,0,0,0,0,0
136039,228729,64,39,I,52860,52756,R,52670,1,86,...,0,0,0,0,0,0,0,0,0,0
136040,228730,64,39,O,52925,52828,I,52756,1,72,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pe = pp.groupby(by=['usuario','practica','modo'])['lec'].sum()
pe = pe.reset_index()
pe

In [ ]:
pc = pp.groupby(by=['usuario','practica','modo'])['lec'].count()
pc = pc.reset_index()
pc = pc.rename(columns={'lec': 'cant'})
pc

,usuario,practica,modo,cant
0,1,1,1,251
1,1,2,1,251
2,1,3,1,251
3,1,4,1,251
4,1,5,1,251
...,...,...,...,...
537,93,26,1,251
538,93,27,1,251
539,93,28,1,251
540,93,29,1,251


In [ ]:
pe = pe.merge(pc, on=['usuario','practica','modo'])
pe

,usuario,practica,modo,lec,cant
0,1,1,1,19,251
1,1,2,1,22,251
2,1,3,1,17,251
3,1,4,1,16,251
4,1,5,1,21,251
...,...,...,...,...,...
537,93,26,1,11,251
538,93,27,1,21,251
539,93,28,1,16,251
540,93,29,1,26,251


In [ ]:
pe['mean_error'] = pe['cant'] / pe['lec']
pe

,usuario,practica,modo,lec,cant,mean_error
0,1,1,1,19,251,13.210526
1,1,2,1,22,251,11.409091
2,1,3,1,17,251,14.764706
3,1,4,1,16,251,15.687500
4,1,5,1,21,251,11.952381
...,...,...,...,...,...,...
537,93,26,1,11,251,22.818182
538,93,27,1,21,251,11.952381
539,93,28,1,16,251,15.687500
540,93,29,1,26,251,9.653846


In [ ]:
pt

,usuario,practica,modo,mean
0,1,1,1,144
1,1,2,1,144
2,1,3,1,145
3,1,4,1,144
4,1,5,1,144
...,...,...,...,...
537,93,26,1,120
538,93,27,1,120
539,93,28,1,112
540,93,29,1,120


In [ ]:
ps

,usuario,practica,modo,std
0,1,1,1,247.187213
1,1,2,1,184.889401
2,1,3,1,178.403024
3,1,4,1,180.920868
4,1,5,1,164.007426
...,...,...,...,...
537,93,26,1,215.701371
538,93,27,1,106.758281
539,93,28,1,121.473611
540,93,29,1,104.778748


In [ ]:
pl

par,usuario,practica,modo, E,",_",.0,.1,.5,.E,04,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,1,0.0,895.5,224.0,320.0,279.5,1185.0,129.0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,2,1,0.0,200.0,353.0,255.0,264.5,1056.0,174.0,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
2,1,3,1,0.0,207.5,241.0,256.0,313.0,974.0,127.0,...,154.0,161.0,446.0,238.0,161.0,224.0,192.0,256.0,192.0,145.0
3,1,4,1,0.0,175.5,224.0,256.0,271.5,1281.0,112.0,...,328.0,176.0,150.0,320.0,129.0,192.0,159.0,288.0,175.5,184.0
4,1,5,1,0.0,176.0,609.0,223.0,464.5,1104.0,176.0,...,177.0,159.0,176.0,239.0,143.0,176.0,194.0,240.0,216.5,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,26,1,0.0,128.0,177.0,233.0,188.5,1088.0,223.0,...,200.0,152.0,176.0,144.0,90.0,154.0,192.0,112.0,123.5,131.5
538,93,27,1,0.0,172.5,224.0,222.0,184.0,664.0,200.0,...,160.0,200.0,128.0,104.0,120.0,152.0,775.0,88.0,163.5,211.5
539,93,28,1,0.0,100.5,176.0,191.0,220.0,696.0,216.0,...,412.0,192.0,127.0,89.0,80.0,129.0,143.0,80.0,132.5,159.5
540,93,29,1,0.0,116.5,175.0,200.0,200.0,872.0,226.0,...,136.0,152.0,168.0,176.0,80.0,144.0,408.0,96.0,140.5,144.0


In [ ]:
final = pe.merge(pt, on=['usuario','practica','modo'])
final = final.merge(pa, on=['usuario','practica','modo'])
final['prom_solapa'] = final['cant'] / final['solapa'] 
final = final.merge(pl, on=['usuario','practica','modo'])

In [ ]:
final = final.drop(columns=['lec','cant','solapa','practica'])
final

,usuario,modo,mean_error,mean,prom_solapa, E,",_",.0,.1,.5,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,13.210526,144,4.048387,0.0,895.5,224.0,320.0,279.5,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,1,11.409091,144,4.403509,0.0,200.0,353.0,255.0,264.5,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
2,1,1,14.764706,145,4.563636,0.0,207.5,241.0,256.0,313.0,...,154.0,161.0,446.0,238.0,161.0,224.0,192.0,256.0,192.0,145.0
3,1,1,15.687500,144,4.048387,0.0,175.5,224.0,256.0,271.5,...,328.0,176.0,150.0,320.0,129.0,192.0,159.0,288.0,175.5,184.0
4,1,1,11.952381,144,4.183333,0.0,176.0,609.0,223.0,464.5,...,177.0,159.0,176.0,239.0,143.0,176.0,194.0,240.0,216.5,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,1,22.818182,120,2.587629,0.0,128.0,177.0,233.0,188.5,...,200.0,152.0,176.0,144.0,90.0,154.0,192.0,112.0,123.5,131.5
538,93,1,11.952381,120,2.221239,0.0,172.5,224.0,222.0,184.0,...,160.0,200.0,128.0,104.0,120.0,152.0,775.0,88.0,163.5,211.5
539,93,1,15.687500,112,2.261261,0.0,100.5,176.0,191.0,220.0,...,412.0,192.0,127.0,89.0,80.0,129.0,143.0,80.0,132.5,159.5
540,93,1,9.653846,120,2.091667,0.0,116.5,175.0,200.0,200.0,...,136.0,152.0,168.0,176.0,80.0,144.0,408.0,96.0,140.5,144.0


In [ ]:
prod = final
prod = prod.where( prod >= 0 , prod['mean'], axis=0)
prod = prod.where( prod != np.inf , 0, axis=0)
prod

,usuario,modo,mean_error,mean,prom_solapa, E,",_",.0,.1,.5,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,13.210526,144,4.048387,0.0,895.5,224.0,320.0,279.5,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,1,11.409091,144,4.403509,0.0,200.0,353.0,255.0,264.5,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
2,1,1,14.764706,145,4.563636,0.0,207.5,241.0,256.0,313.0,...,154.0,161.0,446.0,238.0,161.0,224.0,192.0,256.0,192.0,145.0
3,1,1,15.687500,144,4.048387,0.0,175.5,224.0,256.0,271.5,...,328.0,176.0,150.0,320.0,129.0,192.0,159.0,288.0,175.5,184.0
4,1,1,11.952381,144,4.183333,0.0,176.0,609.0,223.0,464.5,...,177.0,159.0,176.0,239.0,143.0,176.0,194.0,240.0,216.5,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,1,22.818182,120,2.587629,0.0,128.0,177.0,233.0,188.5,...,200.0,152.0,176.0,144.0,90.0,154.0,192.0,112.0,123.5,131.5
538,93,1,11.952381,120,2.221239,0.0,172.5,224.0,222.0,184.0,...,160.0,200.0,128.0,104.0,120.0,152.0,775.0,88.0,163.5,211.5
539,93,1,15.687500,112,2.261261,0.0,100.5,176.0,191.0,220.0,...,412.0,192.0,127.0,89.0,80.0,129.0,143.0,80.0,132.5,159.5
540,93,1,9.653846,120,2.091667,0.0,116.5,175.0,200.0,200.0,...,136.0,152.0,168.0,176.0,80.0,144.0,408.0,96.0,140.5,144.0


In [ ]:
#prod['usuario'] = np.where(prod['usuario']==1, True, False)
prod

,usuario,modo,mean_error,mean,prom_solapa, E,",_",.0,.1,.5,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,13.210526,144,4.048387,0.0,895.5,224.0,320.0,279.5,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,1,11.409091,144,4.403509,0.0,200.0,353.0,255.0,264.5,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
2,1,1,14.764706,145,4.563636,0.0,207.5,241.0,256.0,313.0,...,154.0,161.0,446.0,238.0,161.0,224.0,192.0,256.0,192.0,145.0
3,1,1,15.687500,144,4.048387,0.0,175.5,224.0,256.0,271.5,...,328.0,176.0,150.0,320.0,129.0,192.0,159.0,288.0,175.5,184.0
4,1,1,11.952381,144,4.183333,0.0,176.0,609.0,223.0,464.5,...,177.0,159.0,176.0,239.0,143.0,176.0,194.0,240.0,216.5,512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,93,1,22.818182,120,2.587629,0.0,128.0,177.0,233.0,188.5,...,200.0,152.0,176.0,144.0,90.0,154.0,192.0,112.0,123.5,131.5
538,93,1,11.952381,120,2.221239,0.0,172.5,224.0,222.0,184.0,...,160.0,200.0,128.0,104.0,120.0,152.0,775.0,88.0,163.5,211.5
539,93,1,15.687500,112,2.261261,0.0,100.5,176.0,191.0,220.0,...,412.0,192.0,127.0,89.0,80.0,129.0,143.0,80.0,132.5,159.5
540,93,1,9.653846,120,2.091667,0.0,116.5,175.0,200.0,200.0,...,136.0,152.0,168.0,176.0,80.0,144.0,408.0,96.0,140.5,144.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model        
from sklearn import neighbors           
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import linear_model
X=prod.drop(axis=1,columns='usuario')  
y=prod['usuario']      
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN
    
print('Knn con peso uniforme')
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='distance')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN

print('KNn con un peso en distancia')    
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))



profundidades=np.arange(1,31,1)
p_max=0               
MRSEkNNT_max=0

print("Evaluación modelo Árbol de Decisión en el Validation")

for p in profundidades: 
    modeloT=DecisionTreeClassifier(max_depth=p, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNNT>MRSEkNNT_max:
        p_max=p
        MRSEkNNT_max=MRSEkNNT

print('Valor de k: ', p_max)
print("Mejor MRSE: ", round(MRSEkNNT_max,3))

modelo=linear_model.LogisticRegression(max_iter=1000,penalty='none',fit_intercept=True, random_state=123)
modelo.fit(X_val_train,y_val_train)
y_val_pred=modelo.predict(X_val_test)
AC=accuracy_score(y_val_test, y_val_pred)

print("Regrecion: ")
print("Accuracy: ", round(AC,3))



Knn con peso uniforme
Valor de k:  1
Mejor MRSE:  0.483
KNn con un peso en distancia
Valor de k:  1
Mejor MRSE:  0.483
Evaluación modelo Árbol de Decisión en el Validation
Valor de k:  9
Mejor MRSE:  0.759
Regrecion: 
Accuracy:  0.713


In [ ]:
res = [random.randrange(0, len(prod), 1) for i in range(20)]
res 
for k in res:
    dt = prod.copy(deep=True)
    dt['usuario'] = np.where( dt['usuario']==k , 1 , 0)
    X=dt.drop(axis=1,columns='usuario')  
    y=dt['usuario']      
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
    X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)
    modeloT=DecisionTreeClassifier(max_depth=13, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    #print(str(p) + "  " + str(k))
    print(MRSEkNNT)
    

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
res = prod['usuario'].unique()
res 
top = 0
ton = 0
fn = 0
fp = 0
tn = 0
tp = 0
for k in res:
    dt = prod.copy(deep=True)
    dt['usuario'] = np.where( dt['usuario']==k , 1 , 0)
    X=dt.drop(axis=1,columns='usuario')  
    y=dt['usuario']      
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
    X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)
    modeloT=DecisionTreeClassifier(max_depth=13, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    print(MRSEkNNT)
    y_val_pred = y_val_pred.tolist()
    y_val_test = y_val_test.tolist()
    TP, FP, TN, FN = perf_measure(y_val_pred, y_val_test)
    fn = fn + FN
    fp = fp + FP
    tn = tn + TN
    tp = tp + TP
top = tp + fp
ton = tn + fn
print("total: " + str(t) + " Falso positivo: " + str(fp) + " Falso negativo: " + str(fn) + " Acierto positivo: " + str(tp) + " Acierto negativo: " + str(tn))
print("total: " + str(t) + " Falso positivo: " + str((fp/top)*100) + " Falso negativo: " + str((fn/ton)*100) + " Acierto positivo: " + str((tp/top)*100) + " Acierto negativo: " + str((tn/ton)*100))

0.9425287356321839
0.9770114942528736
0.9540229885057471
0.9540229885057471
0.9310344827586207
0.9655172413793104
0.9655172413793104
0.9425287356321839
0.9885057471264368
0.9540229885057471
0.9655172413793104
0.9655172413793104
0.9885057471264368
0.9310344827586207
0.9425287356321839
1.0
1.0
1.0
0.9770114942528736
1.0
0.9885057471264368
0.9655172413793104
0.9770114942528736
0.9770114942528736
0.9770114942528736
0.9425287356321839
0.9885057471264368
1.0
total: 5016 Falso positivo: 36 Falso negativo: 37 Acierto positivo: 51 Acierto negativo: 2312
total: 5016 Falso positivo: 41.37931034482759 Falso negativo: 1.5751383567475523 Acierto positivo: 58.620689655172406 Acierto negativo: 98.42486164325244


In [ ]:
y_val_test = y_val_pred.tolist()
acierto = 0
falson = 0
falsop = 0
for j in range(len(y_val_pred)):
    pred = int(y_val_pred[j])
    test = int(y_val_test[j])
    if pred == test:
        acierto = acierto + 1
        print(pred)
        print(test)
        print("")
    if ((pred == 1)&(test == 0)):
        falsop = falsop + 1
        print(pred)
        print(test)
        print("")
    if ((pred == 0)&(test == 1)):
        falsop = falsop + 1
        print(pred)
        print(test)
        print("")
print("Segun yo: " + str(acierto) + "  Falso negativo: " + str(falson) + "  Falso positivo: " + str(falsop))

AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
for i in range(len(y_val_pred)):
    print(str(y_val_pred[i]) + "/" + str(y_val_test[i]))

0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
